<a href="https://colab.research.google.com/github/institutohumai/pyeph/blob/main/examples.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>
<div align="center"> Recordá abrir en una nueva pestaña </div>


## *Instalación de la librería*

In [ ]:
!pip install pyeph

In [ ]:
import pyeph

## *Obtención de las bases de datos*

In [ ]:
eph = pyeph.get(data="eph", year=2021, period=2, base_type='individual') 
canastas = pyeph.get(data="canastas") # canasta basica total y alimentaria
adequi = pyeph.get(data="adulto-equivalente") # adulto equivalente

In [ ]:
eph.head(10)

## *Estimación de Pobreza e Indigencia*

In [ ]:
pobreza = pyeph.Poverty(eph,canastas)

### *Pobreza Poblacional*

In [ ]:
# Tasa de pobreza de personas general 
pobreza.population()

In [ ]:
# Pobreza por sexo 
pobreza_sexo = pobreza.population(group_by='CH04')

pobreza_sexo

In [ ]:
pobreza_sexo_etiquetado = pyeph.etiquetar(pobreza_sexo)
pobreza_sexo_etiquetado

### *Pobreza Hogares*

In [ ]:
# Tasa de pobreza hogares 
pobreza.household()

## *Indicadores de mercado laboral*

In [ ]:
mercado_laboral = pyeph.LaborMarket(eph)

### *Desempleo*

In [ ]:
# Tasa de desempleo considerando la población total por sexo
desempleo_sexo = mercado_laboral.unemployment(group_by="CH04", div_by="PT")
desempleo_sexo


In [ ]:
desempleo_sexo_etiquetado = pyeph.etiquetar(desempleo_sexo)
desempleo_sexo_etiquetado

### *Empleo*

In [ ]:
# Tasa de desempleo considerando la Población en Edad de Trabajar
mercado_laboral.employment(div_by="PET")

In [ ]:
# Tasa de empleo considernado la Población Total por sexo
mercado_laboral.employment(group_by="CH04", div_by="PT")

In [ ]:
# Tasa de empleo general considerando la Población en Edad de Trabajar
mercado_laboral.employment(div_by="PET")

### *Actividad*

In [ ]:
# Tasa de actividad considerando la Población Total por sexo
mercado_laboral.activity(group_by="CH04", div_by="PT")